## Data Gathering

In [3]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import random

import time
import csv

import re

import os
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

### Spotrac.com Disabled List Data

In [129]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.spotrac.com/mlb/disabled-list/2015/cumulative-player/pitching/")

In [130]:
with open('dl_list_2015.csv', 'w',newline='') as csvfile:
    file = csv.writer(csvfile)

    file.writerow(['Player', 'Posititon', 'Team', 'Injury', 'Status', 'Days', 'Money_Earned'])
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    rows = soup.find_all(class_='parent')
    rows = rows[61:]

    players = []
    dl_data = []

    for row in rows:
        player = row.find('a').text
        players.append(player)
    
        injury = row.find_all(class_='center')
    
        for info in injury:
            inj = info.text
            dl_data.append(inj)
        
    positions = []
    teams = []
    injuries = []
    status = []
    days = []
    money = []
    

    for index, item in enumerate(dl_data):
        if index%6 == 0:
            positions.append(item)
            
    dl_data = dl_data[1:]

    for index, item in enumerate(dl_data):
        if index%6 == 0:
            teams.append(item)
    
    dl_data = dl_data[1:]

    for index, item in enumerate(dl_data):
        if index%6 == 0:
            injuries.append(item)
            
    dl_data = dl_data[1:]

    for index, item in enumerate(dl_data):
        if index%6 == 0:
            status.append(item)
    
    dl_data = dl_data[1:]
    
    for index, item in enumerate(dl_data):
        if index%6 == 0:
            days.append(item)
    
    dl_data = dl_data[1:]

    for index, item in enumerate(dl_data):
        if index%6 == 0:
            money.append(item)
    
    for num in np.arange(0,240):
            
        observation = [players[num], positions[num], teams[num], injuries[num], status[num], days[num], money[num]]

        file.writerow(observation)

### BrooksBaseball.com Player Card Data

In [5]:
driver = webdriver.Chrome(chromedriver)
driver.get("http://www.brooksbaseball.net/landing.php?player=592091")

In [32]:
playercarddict = {}

for index, row in df.iterrows():
    
    player = df['Name'][index]
    playerid = df['playerid'][index]
    
    page = "http://www.brooksbaseball.net/landing.php?player=" + str(playerid)

    driver.get(page)
    
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        row = soup.find(class_='well').find('p').text
    except:
        row = 0
    
    playercarddict[player] = row

In [36]:
playercarddict

{'A.J. Achter': "Although he has not thrown an MLB pitch in 2019, A.J. Achter threw 1,009 pitches that were tracked by the PITCHf/x system between 2013 and 2016, including pitches thrown in the MLB Regular Season, Spring Training and Fall/Winter Ball. In 2016, he relied primarily on his Change (83mph) and Fourseam Fastball (92mph), also mixing in a Sinker (91mph) and Slider (82mph).BETA Feature:Basic description of 2016 pitches compared to other RHP:His change  has surprising cut action and results in somewhat more groundballs compared to other pitchers' changeups. His fourseam fastball  has less armside movement than typical, has essentially average velo and has some added backspin. His sinker generates more whiffs/swing compared to other pitchers' sinkers, results in more flyballs compared to other pitchers' sinkers and has little sinking action compared to a true sinker. His slider is basically never swung at and missed compared to other pitchers' sliders, has much less depth than e

In [154]:
r = re.compile('Slider.{0,20}\((\d*mph)\)')
slidersdict = {}

for key, value in playercarddict.items():
    try:
        pitch = r.search(value)
        speed = pitch.group(1)
        slidersdict[key] = speed
    except:
        slidersdict[key] = 0

### Pitches of Interest

- Slider
- Sinker
- Knuckle
- Curve
- Fourseam Fastball
- Change
- Cutter
- Splitter

In [324]:
pitches = fastballs.join(sliders['Slider'])
pitches = pitches.join(sinkers['Sinker'])
pitches = pitches.join(knuckles['Knuckle'])
pitches = pitches.join(curves['Curve'])
pitches = pitches.join(changeups['Change'])
pitches = pitches.join(cutters['Cutter'])
pitches = pitches.join(splitters['Splitter'])

In [325]:
pitches.head()

,Name,Fastball,Slider,Sinker,Knuckle,Curve,Change,Cutter,Splitter
0,A.J. Achter,92mph,82mph,82mph,0,0,83mph,0,0
1,A.J. Burnett,85mph,0,91mph,82mph,82mph,85mph,0,0
2,A.J. Cole,80mph,85mph,93mph,0,80mph,87mph,0,0
3,A.J. Griffin,67mph,0,86mph,0,67mph,84mph,84mph,0
4,A.J. Minter,92mph,92mph,0,0,0,87mph,0,0


In [326]:
pitches['Fastball'] = pitches['Fastball'].str.replace('mph', "")
pitches['Slider'] = pitches['Slider'].str.replace('mph', "")
pitches['Sinker'] = pitches['Sinker'].str.replace('mph', "")
pitches['Knuckle'] = pitches['Knuckle'].str.replace('mph', "")
pitches['Curve'] = pitches['Curve'].str.replace('mph', "")
pitches['Change'] = pitches['Change'].str.replace('mph', "")
pitches['Cutter'] = pitches['Cutter'].str.replace('mph', "")
pitches['Splitter'] = pitches['Splitter'].str.replace('mph', "")

In [327]:
pitches.head()

,Name,Fastball,Slider,Sinker,Knuckle,Curve,Change,Cutter,Splitter
0,A.J. Achter,92,82,82,NaN,NaN,83,NaN,NaN
1,A.J. Burnett,85,NaN,91,82,82,85,NaN,NaN
2,A.J. Cole,80,85,93,NaN,80,87,NaN,NaN
3,A.J. Griffin,67,NaN,86,NaN,67,84,84,NaN
4,A.J. Minter,92,92,NaN,NaN,NaN,87,NaN,NaN


In [328]:
pitches['Fastball'] = pd.to_numeric(pitches['Fastball'])
pitches['Slider'] = pd.to_numeric(pitches['Slider'])
pitches['Sinker'] = pd.to_numeric(pitches['Sinker'])
pitches['Knuckle'] = pd.to_numeric(pitches['Knuckle'])
pitches['Curve'] = pd.to_numeric(pitches['Curve'])
pitches['Change'] = pd.to_numeric(pitches['Change'])
pitches['Cutter'] = pd.to_numeric(pitches['Cutter'])
pitches['Splitter'] = pd.to_numeric(pitches['Splitter'])

In [329]:
pitches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1507 entries, 0 to 1506
Data columns (total 9 columns):
Name        1507 non-null object
Fastball    1327 non-null float64
Slider      1017 non-null float64
Sinker      787 non-null float64
Knuckle     87 non-null float64
Curve       779 non-null float64
Change      1059 non-null float64
Cutter      342 non-null float64
Splitter    132 non-null float64
dtypes: float64(8), object(1)
memory usage: 106.0+ KB


In [330]:
pitches['Max_Speed'] = pitches.max(axis=1)

In [331]:
pitches.head()

,Name,Fastball,Slider,Sinker,Knuckle,Curve,Change,Cutter,Splitter,Max_Speed
0,A.J. Achter,92.0,82.0,82.0,NaN,NaN,83.0,NaN,NaN,92.0
1,A.J. Burnett,85.0,NaN,91.0,82.0,82.0,85.0,NaN,NaN,91.0
2,A.J. Cole,80.0,85.0,93.0,NaN,80.0,87.0,NaN,NaN,93.0
3,A.J. Griffin,67.0,NaN,86.0,NaN,67.0,84.0,84.0,NaN,86.0
4,A.J. Minter,92.0,92.0,NaN,NaN,NaN,87.0,NaN,NaN,92.0


In [333]:
pitches = pitches.fillna(0)
pitches1 = pitches.drop(['Name','Max_Speed'],axis=1)
pitches1[pitches1 != 0] = 1

In [335]:
pitches = pitches[['Name','Max_Speed']]

,Name,Max_Speed
0,A.J. Achter,92.0
1,A.J. Burnett,91.0
2,A.J. Cole,93.0
3,A.J. Griffin,86.0
4,A.J. Minter,92.0


In [336]:
pitches = pitches.join(pitches1)

In [337]:
pitches.head()

,Name,Max_Speed,Fastball,Slider,Sinker,Knuckle,Curve,Change,Cutter,Splitter
0,A.J. Achter,92.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,A.J. Burnett,91.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
2,A.J. Cole,93.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
3,A.J. Griffin,86.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
4,A.J. Minter,92.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [338]:
pitches.to_pickle("./pitches.pkl")